In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

- kaggle dataset (초안)

In [146]:
book_df=pd.read_csv('C:\\Workspaces\\Bookitoki\\YJ\\data\\book_review_data.csv')

In [147]:
print(book_df.columns)
print(book_df.shape)

Index(['Unnamed: 0', 'user_id', 'location', 'age', 'isbn', 'rating',
       'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_s', 'img_m', 'img_l', 'Summary', 'Language', 'Category', 'city',
       'state', 'country'],
      dtype='object')
(1031175, 19)


In [148]:
# 1) 필요한 컬럼 도출
# 2) 형변환 (float > int)

book_df=book_df[['user_id',"age","rating","book_title",'book_author','year_of_publication','img_m','Summary','Category','country','Language']]
book_df[['age',"year_of_publication"]]=book_df[['age',"year_of_publication"]].astype(int)

In [149]:
# 최소 평가 개수 ( 평가가 30개보다 많은 책만 추출 )
counts=book_df['book_title'].value_counts()
counts.head()

book_title
Wild Animus                   2502
The Lovely Bones: A Novel     1295
The Da Vinci Code              898
A Painted House                838
The Nanny Diaries: A Novel     828
Name: count, dtype: int64

In [150]:
title = counts[counts > 30].index
title[:5]

Index(['Wild Animus', 'The Lovely Bones: A Novel', 'The Da Vinci Code',
       'A Painted House', 'The Nanny Diaries: A Novel'],
      dtype='object', name='book_title')

In [151]:
book_filter_df=book_df[book_df['book_title'].isin(title)]

In [152]:
# 최소 평가 개수 ( 평가가 20개보다 많은 사용자만 추출 )
counts=book_filter_df["user_id"].value_counts()
user_id_=counts[counts>10].index
book_filter_df = book_filter_df[book_filter_df['user_id'].isin(user_id_)]

In [153]:
# 무의미한 0점 평가 제거
rating_zero=book_filter_df[book_filter_df['rating']==0].index
book_filter_df.drop(rating_zero, axis=0, inplace=True)

In [154]:
# 한 카테고리안에 책이 5권 미만인 경우 제거
counts=book_filter_df["Category"].value_counts()
categories = counts[counts > 5].index
book_filter_df = book_filter_df[book_filter_df['Category'].isin(categories)]
idx=book_filter_df[book_filter_df["Summary"]==9].index
book_filter_df.drop(idx,axis=0,inplace=True)

In [155]:
print(book_filter_df.shape)
len(book_filter_df['user_id'].unique()), len(book_filter_df['book_title'].unique())

(84280, 11)


(4920, 4437)

In [156]:
book_filter_df["book_title"]=book_filter_df['book_title'].str.lower()

In [157]:
user_book_df=book_filter_df.pivot_table(index='user_id',columns='book_title',values='rating',fill_value=0)
print(user_book_df.shape) # user간 유사도 구할거 아니니까
book_user_df=user_book_df.T



(4920, 4401)


In [158]:
user_book_df.iloc[55].sort_values(ascending=False)[:10]

book_title
it                                                                          8.0
buddha of suburbia                                                          8.0
christine                                                                   8.0
thinner                                                                     8.0
practical demonkeeping                                                      7.0
firestarter (signet book)                                                   7.0
dolores claiborne                                                           7.0
cujo                                                                        6.0
something wicked this way comes                                             6.0
stupid white men ...and other sorry excuses for the state of the nation!    6.0
Name: 4334, dtype: float64

In [159]:
from sklearn.metrics.pairwise import cosine_similarity

book_sim = cosine_similarity(book_user_df, book_user_df)
print(book_sim.shape)

book_sim_df = pd.DataFrame(book_sim, index=user_book_df.columns, columns=user_book_df.columns)w

(4401, 4401)


In [160]:
# 전체 가중평점 예측하기 (4920,4401)

def predict_ratings(book_user_df, book_sim_df):


 return user_book_df.dot(book_sim_df) / np.abs(book_sim_df).sum(axis=1)
ratings_pred_df = predict_ratings(user_book_df, book_sim_df)
print(ratings_pred_df.shape)
ratings_pred_df.head(1).T




(4920, 4401)


user_id,243
book_title,
'salem's lot,0.026268
10 lb. penalty,0.015752
101 dalmatians,0.044246
16 lighthouse road,0.010331
1984,0.033637
...,...
zen and the art of motorcycle maintenance: an inquiry into values,0.044673
zia,0.000000
zlata's diary: a child's life in sarajevo,0.032801


In [161]:
#실제 평점과 예측평점 오차 비교

from sklearn.metrics import mean_squared_error

def get_mse(actual, pred):
    non_zero_idx = actual.nonzero()
    
    actual = actual[non_zero_idx]
    pred = pred[non_zero_idx]
    return mean_squared_error(actual, pred)
get_mse(user_book_df.values, ratings_pred_df.values)

54.54519201221758

In [ ]:
#  모델 실험


In [163]:
def predict_ratings_by_user_book(user_book_idx,movie_idx, topn_sim_idx):
    #영화별 평점 유사도
    topn_sim = book_sim[book_idx,:][topn_sim_idx]
    # 평점유사도 topn건에 대한 사용점 
    topn_rating = user_book_df.values[user_idx,:][topn_sim_idx
    ]
    return topn_sim.dot(topn_rating) / np.abs(topn_sim).sum()
topn = 20 
user_idx = 135
book_idx = 33
topn_sim_idx = book_sim_df.iloc[book_idx].argsort()[:-(topn + 1) :-1]
predict_ratings_by_user_book(user_idx,book_idx,topn_sim_idx)

0.0

In [164]:
from sklearn.metrics.pairwise import cosine_similarity                     

book_sim = cosine_similarity(book_user_df, book_user_df)
#print(book_sim.shape)
book_sim[:5, :100]

book_sim_df = pd.DataFrame(book_sim, index=book_user_df.index, columns=book_user_df.index)
print(book_sim_df.shape)

(4401, 4401)


In [165]:
def predict_ratings(topn=10):
    pred = np.zeros(user_book_df.shape)
    for book_idx in range(pred.shape[1]):
        topn_sim_idx = book_sim_df.iloc[book_idx].argsort()[:-(topn + 1):-1]
    for user_idx in range(pred.shape[0]):
        pred[user_idx, book_idx] = predict_ratings_by_user_book(user_idx, book_idx, topn_sim_idx)
        
    return pred

ratings_pred = predict_ratings(20)
print(ratings_pred.shape)

(4920, 4401)


In [192]:
# 예측평점 df
ratings_pred_df = pd.DataFrame(ratings_pred, index= user_book_df.index, columns = user_book_df.columns)
ratings_pred_df.head(87).T

user_id,243,254,383,388,487,503,507,638,735,741,...,6333,6345,6431,6532,6537,6543,6563,6566,6575,6577
book_title,,,,,,,,,,,,,,,,,,,,,
'salem's lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 lb. penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 dalmatians,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 lighthouse road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zen and the art of motorcycle maintenance: an inquiry into values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zlata's diary: a child's life in sarajevo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [167]:
#특정사용자에게 영화 추천
book_df.iloc[(ratings_pred[162].argsort()[:-11:-1])]

,user_id,age,rating,book_title,book_author,year_of_publication,img_m,Summary,Category,country,Language
4400,119341,33,8,To Kill a Mockingbird,Harper Lee,1988,http://images.amazon.com/images/P/0446310786.0...,The unforgettable novel of a childhood in a sl...,['Fiction'],australia,en
1468,77480,51,2,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en
1462,76436,24,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],portugal,en
1463,76901,16,7,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],canada,en
1464,76942,25,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en
1465,76968,21,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],germany,en
1466,77181,33,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en
1467,77332,58,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en
1469,77809,29,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en
1443,74640,28,0,Wild Animus,Rich Shapero,2004,http://images.amazon.com/images/P/0971880107.0...,"Wild animus is a search for the primordial, a ...",['Fiction'],usa,en


In [168]:
# 사용자별 안본 영화 조회

def get_unseen_book(user_idx):
    user_ratings_df = user_book_df.iloc[user_idx]
    return user_ratings_df[user_ratings_df == 0]
'1984' in get_unseen_book(162)

True

In [169]:
def recommend_book(user_idx, topn=10):
    # 안 본 책을 찾기
    unseen = get_unseen_book(user_idx).index
    
    # 예측 평점 상위 topn 개의 책 추천
    temp = ratings_pred_df.loc[user_idx, unseen].sort_values(ascending=False)[:topn]
    
    # 사용자 평점 조회
    user_ratings_df = user_book_df.loc[user_idx, temp.index]

    # 추천 결과 반환
    return pd.DataFrame({
        'title': temp.index,
        'pred_rating': temp.values,
        'user_rating': user_ratings_df.values
    })

# 추천 함수 호출
recommend_book(23)

KeyError: 23

In [170]:
# 사용자별 영화 추천
'''
def recommend_book(user_idx, topn=10):
    #안본 책 (series 반환 후 , index 책 제목을 label로 변환)
    unseen = get_unseen_book(user_idx).index
# 예측 평점 상위 topn 개의 영화 추천
    temp = ratings_pred_df.loc[user_idx, unseen].sort_values(ascending=False)[:topn]
# 사용자 평점 조회 ( 모두 안본 책 확인):
# loc는 라벨조회 이므로, 올바른 user ID 조회를 위해 
# user_idx + 1로 처리함
    
    user_ratings_df = user_book_df.loc[user_idx + 1, temp.index]

    return pd.DataFrame({
        'title': temp.index,
        'pred_rating': temp.values,
        'user_rating': user_ratings_df.values
    })

recommend_book(23)'''

"\ndef recommend_book(user_idx, topn=10):\n    #안본 책 (series 반환 후 , index 책 제목을 label로 변환)\n    unseen = get_unseen_book(user_idx).index\n# 예측 평점 상위 topn 개의 영화 추천\n    temp = ratings_pred_df.loc[user_idx, unseen].sort_values(ascending=False)[:topn]\n# 사용자 평점 조회 ( 모두 안본 책 확인):\n# loc는 라벨조회 이므로, 올바른 user ID 조회를 위해 \n# user_idx + 1로 처리함\n    \n    user_ratings_df = user_book_df.loc[user_idx + 1, temp.index]\n\n    return pd.DataFrame({\n        'title': temp.index,\n        'pred_rating': temp.values,\n        'user_rating': user_ratings_df.values\n    })\n\nrecommend_book(23)"

In [195]:
# 특정 도서의 상위 평점 도서 조회

def get_user_topn_book(user_idx, topn=10):
    return user_book_df.iloc[user_idx].sort_values(ascending=False)[:topn]
get_user_topn_book(33)


book_title
hinds' feet on high places                         10.0
number the stars (yearling newbery)                10.0
atlas shrugged                                      9.0
the mists of avalon                                 9.0
divine secrets of the ya-ya sisterhood: a novel     9.0
the good earth (enriched classics)                  8.0
the fountainhead                                    8.0
the secret life of bees                             8.0
skipping christmas                                  7.0
to the nines: a stephanie plum novel                7.0
Name: 2337, dtype: float64